## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-25-08-53-39 +0000,nypost,US confirms nation’s first travel-associated h...,https://nypost.com/2025/08/25/us-news/us-confi...
1,2025-08-25-08-48-43 +0000,nyt,Keurig Dr Pepper to Acquire Peet’s in $18 Bill...,https://www.nytimes.com/2025/08/25/business/ke...
2,2025-08-25-08-44-07 +0000,nyt,China Evergrande Delisted From Hong Kong Stock...,https://www.nytimes.com/2025/08/24/business/ch...
3,2025-08-25-08-41-24 +0000,nyt,"5 Years On, China’s Property Crisis Has No End...",https://www.nytimes.com/2025/08/25/business/ch...
4,2025-08-25-08-30-00 +0000,wsj,German Business Sentiment Keeps Improving Desp...,https://www.wsj.com/economy/german-business-se...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2323/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
69,trump,19
101,new,12
171,ukraine,11
489,russia,9
472,year,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
111,2025-08-24-18-08-34 +0000,nypost,Russia’s top diplomat says Zelensky can’t sign...,https://nypost.com/2025/08/24/world-news/russi...,58
131,2025-08-24-14-39-41 +0000,nypost,Trump announces he’ll attend Ryder Cup in New ...,https://nypost.com/2025/08/24/us-news/trump-an...,55
142,2025-08-24-14-05-19 +0000,nypost,Russia’s foreign minister throws up a new cond...,https://nypost.com/2025/08/24/us-news/russias-...,54
110,2025-08-24-18-08-34 +0000,nypost,Vance reveals what Putin has conceded in Ukrai...,https://nypost.com/2025/08/24/us-news/vance-re...,50
163,2025-08-24-10-00-00 +0000,latimes,California's lightning-fast push for partisan ...,https://www.latimes.com/politics/story/2025-08...,49


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
111,58,2025-08-24-18-08-34 +0000,nypost,Russia’s top diplomat says Zelensky can’t sign...,https://nypost.com/2025/08/24/world-news/russi...
163,49,2025-08-24-10-00-00 +0000,latimes,California's lightning-fast push for partisan ...,https://www.latimes.com/politics/story/2025-08...
107,36,2025-08-24-18-30-00 +0000,nypost,Australian mom accused of killing American ste...,https://nypost.com/2025/08/24/world-news/lisa-...
142,28,2025-08-24-14-05-19 +0000,nypost,Russia’s foreign minister throws up a new cond...,https://nypost.com/2025/08/24/us-news/russias-...
8,26,2025-08-25-07-39-54 +0000,nypost,Four young suspects charged in Bronx shooting ...,https://nypost.com/2025/08/25/us-news/four-you...
35,23,2025-08-25-02-00-00 +0000,wsj,Demand for workers has cooled yet the unemploy...,https://www.wsj.com/economy/immigration-worker...
122,22,2025-08-24-16-41-00 +0000,wsj,President Trump’s attempts to control monetary...,https://www.wsj.com/economy/central-banking/fe...
36,22,2025-08-25-01-56-00 +0000,wsj,Here’s what you need to know about the unusual...,https://www.wsj.com/politics/policy/texas-cali...
39,22,2025-08-25-01-35-57 +0000,nypost,Freak hot tub incident almost kills 2 elderly ...,https://nypost.com/2025/08/24/us-news/freak-ho...
87,21,2025-08-24-20-51-10 +0000,nypost,Israeli airstrikes hit Yemen’s capital after H...,https://nypost.com/2025/08/24/world-news/israe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
